In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.impute import SimpleImputer

In [50]:
historical_weather = pd.read_csv(r"C:\0A___________________________________\Predictor 1.0\historical_weather.csv")
test_data = pd.read_csv(r"C:\0A___________________________________\Predictor 1.0\submission_key.csv")

In [51]:
historical_weather.drop(columns=['snow_depth_mm'], inplace=True)

In [52]:
historical_weather['date'] = pd.to_datetime(historical_weather['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

In [53]:
historical_weather['city_id'] = historical_weather['city_id'].str.extract('(\d+)').astype(int)
test_data['city_id'] = test_data['city_id'].str.extract('(\d+)').astype(int)

In [54]:
imputer = SimpleImputer(strategy='mean')

In [55]:
historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']] = imputer.fit_transform(historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])

In [56]:
historical_weather['temp_range'] = historical_weather['max_temp_c'] - historical_weather['min_temp_c']
historical_weather['rolling_range_temp'] = historical_weather['temp_range'].rolling(window=7).mean()
historical_weather['avg_temp_rel'] = (historical_weather['avg_temp_c'] - historical_weather['min_temp_c']) / (historical_weather['max_temp_c'] - historical_weather['min_temp_c'])
historical_weather['ema_temp'] = historical_weather['avg_temp_c'].ewm(span=7, adjust=False).mean()
historical_weather['cum_precip'] = historical_weather['precipitation_mm'].cumsum()
historical_weather['temp_precip_interaction'] = historical_weather['avg_temp_c'] * historical_weather['precipitation_mm']
historical_weather['temp_wind_interaction'] = historical_weather['avg_temp_c'] * historical_weather['avg_wind_speed_kmh']

# Temperature anomalies
historical_weather['temp_anomaly'] = historical_weather['avg_temp_c'] - historical_weather['avg_temp_c'].mean()

# Precipitation anomalies
historical_weather['precip_anomaly'] = historical_weather['precipitation_mm'] - historical_weather['precipitation_mm'].mean()

# Complex interactions
historical_weather['temp_precip_wind_interaction'] = historical_weather['avg_temp_c'] * historical_weather['precipitation_mm'] * historical_weather['avg_wind_speed_kmh']

In [57]:
historical_weather['rolling_mean_temp'] = historical_weather['avg_temp_c'].rolling(window=7).mean()
historical_weather['rolling_std_temp'] = historical_weather['avg_temp_c'].rolling(window=7).std()
historical_weather['rolling_sum_precip'] = historical_weather['precipitation_mm'].rolling(window=7).sum()
historical_weather['rolling_std_precip'] = historical_weather['precipitation_mm'].rolling(window=7).sum()

historical_weather['rolling_mean_temp_14'] = historical_weather['avg_temp_c'].rolling(window=14).mean()
historical_weather['rolling_std_temp_14'] = historical_weather['avg_temp_c'].rolling(window=14).std()
historical_weather['rolling_sum_precip_14'] = historical_weather['precipitation_mm'].rolling(window=14).sum()
historical_weather['rolling_std_precip_14'] = historical_weather['precipitation_mm'].rolling(window=14).sum()


historical_weather['rolling_mean_temp_30'] = historical_weather['avg_temp_c'].rolling(window=30).mean()
historical_weather['rolling_std_temp_30'] = historical_weather['avg_temp_c'].rolling(window=30).std()
historical_weather['rolling_sum_precip_30'] = historical_weather['precipitation_mm'].rolling(window=30).sum()
historical_weather['rolling_std_precip_30'] = historical_weather['precipitation_mm'].rolling(window=30).sum()

In [58]:
historical_weather['temp_deviation'] = historical_weather['avg_temp_c'] - historical_weather['avg_temp_c'].rolling(window=30).mean()
# Day-to-day temperature change
historical_weather['temp_day_change'] = historical_weather['avg_temp_c'] - historical_weather['avg_temp_c'].shift(1)
    
# Heavy precipitation indicator
historical_weather['heavy_precip'] = (historical_weather['precipitation_mm'] > 10).astype(int)
    
# # Wind speed categories (example bins: calm, breeze, gale)
# bins = [0, 5, 15, 30, np.inf]
# labels = ['Calm', 'Breeze', 'Gale', 'Storm']
# category_dict = {'Calm': 1, 'Breeze': 2, 'Gale': 3, 'Storm': 4}
# historical_weather['wind_speed_category'] = pd.cut(historical_weather['avg_wind_speed_kmh'], bins=bins, labels=labels)
# historical_weather['wind_speed_category_numeric'] = historical_weather['wind_speed_category'].map(category_dict)

# Wind direction sine and cosine components
historical_weather['wind_dir_sin'] = np.sin(np.radians(historical_weather['avg_wind_dir_deg']))
historical_weather['wind_dir_cos'] = np.cos(np.radians(historical_weather['avg_wind_dir_deg']))

# Temperature percentile rank
historical_weather['temp_percentile'] = historical_weather['avg_temp_c'].rank(pct=True)

# Cumulative moving average
historical_weather['cum_avg_temp'] = historical_weather['avg_temp_c'].expanding().mean()

In [59]:
from scipy.stats import skew, kurtosis
# Statistical aggregates
historical_weather['mean_temp_c_month'] = historical_weather.groupby(historical_weather['date'].dt.month)['avg_temp_c'].transform('mean')
historical_weather['max_temp_c_month'] = historical_weather.groupby(historical_weather['date'].dt.month)['max_temp_c'].transform('max')
historical_weather['min_temp_c_month'] = historical_weather.groupby(historical_weather['date'].dt.month)['min_temp_c'].transform('min')
historical_weather['variance_temp_c_month'] = historical_weather.groupby(historical_weather['date'].dt.month)['avg_temp_c'].transform('var')
historical_weather['skewness_temp_c'] = historical_weather['avg_temp_c'].rolling(window=30).apply(skew, raw=True)
historical_weather['kurtosis_temp_c'] = historical_weather['avg_temp_c'].rolling(window=30).apply(kurtosis, raw=True)

# Event features
historical_weather['heat_wave_duration'] = historical_weather['avg_temp_c'].apply(lambda x: 1 if x > 30 else 0).rolling(window=3).sum()
historical_weather['cold_wave_duration'] = historical_weather['avg_temp_c'].apply(lambda x: 1 if x < 0 else 0).rolling(window=3).sum()
historical_weather['precipitation_event_duration'] = historical_weather['precipitation_mm'].apply(lambda x: 1 if x > 10 else 0).rolling(window=3).sum()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\window\rolling.py:1535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return window_func(values, begin, end, min_periods)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\window\rolling.py:1535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return window_func(values, begin, end, min_periods)


In [60]:
day_of_year = historical_weather['date'].dt.dayofyear
historical_weather['fourier_sin'] = np.sin(2 * np.pi * day_of_year / 365.25)
historical_weather['fourier_cos'] = np.cos(2 * np.pi * day_of_year / 365.25)

In [61]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['season_sin_month'] = np.sin(2 * np.pi * df['date'].dt.month / 12)
    df['season_cos_month'] = np.cos(2 * np.pi * df['date'].dt.month / 12)
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['season_sin_day'] = np.sin(2 * np.pi * df['date'].dt.day / 31)
    df['season_cos_day'] = np.cos(2 * np.pi * df['date'].dt.day / 31)
    df['weekofyear'] = df['date'].dt.isocalendar().week
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300], 
                          labels=['Spring', 'Summer', 'Fall', 'Winter'])
    df['season'] = df['season'].cat.codes
    return df

historical_weather = create_features(historical_weather)

In [62]:
historical_weather['seasonal_mean_temp'] = historical_weather.groupby('season')['avg_temp_c'].transform('mean')

In [63]:
def create_lag_features(df, lags, cols):
    for col in cols:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df.groupby('city_id')[col].shift(lag)
    return df

In [64]:
lag_features = ['avg_temp_c']
lags = [1, 2, 3, 4, 5, 6, 7]  # 1-day, 2-day, 3-day, 7-day, 14-day, and 30-day lags
historical_weather = create_lag_features(historical_weather, lags, lag_features)

lags_1 = [1,3,7]
lag_features_1 = ['min_temp_c','max_temp_c','precipitation_mm','avg_wind_dir_deg']
historical_weather = create_lag_features(historical_weather, lags_1, lag_features_1)

In [65]:
historical_weather.columns

Index(['city_id', 'date', 'avg_temp_c', 'min_temp_c', 'max_temp_c',
       'precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh',
       'temp_range', 'rolling_range_temp', 'avg_temp_rel', 'ema_temp',
       'cum_precip', 'temp_precip_interaction', 'temp_wind_interaction',
       'temp_anomaly', 'precip_anomaly', 'temp_precip_wind_interaction',
       'rolling_mean_temp', 'rolling_std_temp', 'rolling_sum_precip',
       'rolling_std_precip', 'rolling_mean_temp_14', 'rolling_std_temp_14',
       'rolling_sum_precip_14', 'rolling_std_precip_14',
       'rolling_mean_temp_30', 'rolling_std_temp_30', 'rolling_sum_precip_30',
       'rolling_std_precip_30', 'temp_deviation', 'temp_day_change',
       'heavy_precip', 'wind_dir_sin', 'wind_dir_cos', 'temp_percentile',
       'cum_avg_temp', 'mean_temp_c_month', 'max_temp_c_month',
       'min_temp_c_month', 'variance_temp_c_month', 'skewness_temp_c',
       'kurtosis_temp_c', 'heat_wave_duration', 'cold_wave_duration',
       'precipit

In [66]:
historical_weather

,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,avg_wind_dir_deg,avg_wind_speed_kmh,temp_range,rolling_range_temp,...,min_temp_c_lag_7,max_temp_c_lag_1,max_temp_c_lag_3,max_temp_c_lag_7,precipitation_mm_lag_1,precipitation_mm_lag_3,precipitation_mm_lag_7,avg_wind_dir_deg_lag_1,avg_wind_dir_deg_lag_3,avg_wind_dir_deg_lag_7
0,1,2014-01-01,6.6,-1.4,11.6,3.679964,168.000000,6.2,13.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2014-01-02,9.3,6.3,13.3,3.679964,155.000000,10.0,7.0,NaN,...,NaN,11.6,NaN,NaN,3.679964,NaN,NaN,168.000000,NaN,NaN
2,1,2014-01-03,7.6,1.9,14.0,3.679964,175.650289,5.8,12.1,NaN,...,NaN,13.3,NaN,NaN,3.679964,NaN,NaN,155.000000,NaN,NaN
3,1,2014-01-04,7.6,3.9,13.3,3.679964,291.000000,11.3,9.4,NaN,...,NaN,14.0,11.6,NaN,3.679964,3.679964,NaN,175.650289,168.0,NaN
4,1,2014-01-05,8.6,0.5,16.9,3.679964,175.650289,5.0,16.4,NaN,...,NaN,13.3,13.3,NaN,3.679964,3.679964,NaN,291.000000,155.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182333,112,2018-12-27,22.0,15.9,27.9,0.000000,100.000000,13.2,12.0,10.085714,...,17.9,28.6,26.5,25.3,0.000000,0.000000,1.3,68.000000,28.0,356.0
182334,112,2018-12-28,21.9,14.9,28.2,0.000000,91.000000,12.6,13.3,11.114286,...,17.1,27.9,28.0,23.2,0.000000,0.000000,20.9,100.000000,36.0,121.0
182335,112,2018-12-29,22.4,16.3,28.2,0.000000,61.000000,14.2,11.9,11.885714,...,15.9,28.2,28.6,22.4,0.000000,0.000000,1.7,91.000000,68.0,137.0
182336,112,2018-12-30,21.6,18.5,26.6,1.600000,70.000000,17.0,8.1,11.600000,...,14.2,28.2,27.9,24.3,0.000000,0.000000,0.2,61.000000,100.0,122.0


In [67]:
historical_weather = historical_weather.dropna()

In [68]:
# scaler = StandardScaler()
# scaler = scaler.fit(historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])
# historical_weather_scaled = scaler.transform(historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])

In [69]:
# def prepare_data_for_lstm(data, n_past=14, n_future=7):
#     X, Y = [], []
#     for i in range(n_past, len(data) - n_future + 1):
#         X.append(data[i - n_past:i, :])  # past days data
#         Y.append(data[i:i + n_future, 0])  # future days target (avg_temp_c)
#     return np.array(X), np.array(Y)

# trainX, trainY = prepare_data_for_lstm(historical_weather_scaled)

In [70]:
# # Define the LSTM model
# model = Sequential()
# model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
# model.add(LSTM(32, activation='relu', return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(trainY.shape[1]))  # Output layer for predicting avg_temp_c

# model.compile(optimizer='adam', loss='mse')
# model.summary()

In [71]:
# history = model.fit(trainX, trainY, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

In [72]:
# plt.plot(history.history['loss'], label='Training loss')
# plt.plot(history.history['val_loss'], label='Validation loss')
# plt.legend()

In [73]:
# predictions = []
# for index, row in test_data.iterrows():
#     city_id = row['city_id']
#     date = row['date']
#     city_data = historical_weather[(historical_weather['city_id'] == city_id) & (historical_weather['date'] <= date)]
#     city_data_scaled = scaler.transform(city_data[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])
    
#     # Reshape data for LSTM input
#     X = np.expand_dims(city_data_scaled[-14:, :], axis=0)  # Using last 14 days data
#     prediction = model.predict(X)
#     predictions.append(prediction[0][0])  # Predicted avg_temp_c for the date

# # Create submission dataframe
# submission = pd.DataFrame({
#     'submission_ID': test_data['submission_ID'],
#     'avg_temp_c': predictions
# })

# # Save submission to CSV file
# submission.to_csv('my_submission.csv', index=False)

In [75]:
# Load and preprocess the data
# df = pd.read_csv('/mnt/data/historical_weather.csv')
# df['date'] = pd.to_datetime(df['date'])
# df = create_features(df)

from sklearn.preprocessing import MinMaxScaler

# Function to prepare data for LSTM
def prepare_lstm_data(df, city_id, sequence_length=30):
    city_df = df[df['city_id'] == city_id].sort_values('date')
    features = ['avg_temp_c', 'min_temp_c', 'max_temp_c',
       'precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh',
       'temp_range', 'rolling_range_temp', 'avg_temp_rel', 'ema_temp',
       'cum_precip', 'temp_precip_interaction', 'temp_wind_interaction',
       'temp_anomaly', 'precip_anomaly', 'temp_precip_wind_interaction',
       'rolling_mean_temp', 'rolling_std_temp', 'rolling_sum_precip',
       'rolling_std_precip', 'rolling_mean_temp_14', 'rolling_std_temp_14',
       'rolling_sum_precip_14', 'rolling_std_precip_14',
       'rolling_mean_temp_30', 'rolling_std_temp_30', 'rolling_sum_precip_30',
       'rolling_std_precip_30', 'temp_deviation', 'temp_day_change',
       'heavy_precip','wind_dir_sin', 'wind_dir_cos',
       'temp_percentile', 'cum_avg_temp', 'mean_temp_c_month',
       'max_temp_c_month', 'min_temp_c_month', 'variance_temp_c_month',
       'skewness_temp_c', 'kurtosis_temp_c', 'heat_wave_duration',
       'cold_wave_duration', 'precipitation_event_duration', 'fourier_sin',
       'fourier_cos', 'dayofweek', 'quarter', 'month', 'season_sin_month',
       'season_cos_month', 'year', 'dayofyear', 'dayofmonth', 'season_sin_day',
       'season_cos_day', 'weekofyear', 'date_offset', 'season',
       'seasonal_mean_temp', 'avg_temp_c_lag_1', 'avg_temp_c_lag_2',
       'avg_temp_c_lag_3', 'avg_temp_c_lag_4', 'avg_temp_c_lag_5',
       'avg_temp_c_lag_6', 'avg_temp_c_lag_7', 'min_temp_c_lag_1',
       'min_temp_c_lag_3', 'min_temp_c_lag_7', 'max_temp_c_lag_1',
       'max_temp_c_lag_3', 'max_temp_c_lag_7', 'precipitation_mm_lag_1',
       'precipitation_mm_lag_3', 'precipitation_mm_lag_7',
       'avg_wind_dir_deg_lag_1', 'avg_wind_dir_deg_lag_3',
       'avg_wind_dir_deg_lag_7']
    city_df = city_df[features].dropna()
    
    scaler = MinMaxScaler()
    city_scaled = scaler.fit_transform(city_df)
    
    X, y = [], []
    for i in range(sequence_length, len(city_scaled) - 7):
        X.append(city_scaled[i-sequence_length:i])
        y.append(city_scaled[i:i+7, 0])  # Predict next 7 days of avg_temp_c

    X, y = np.array(X), np.array(y)
    return X, y, scaler

# Function to build and train LSTM model
def train_lstm(X_train, y_train):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(7))  # Predicting 7 days
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)
    return model

# Function to make predictions
def predict_next_7_days(model, X, scaler, num_features):
    prediction_scaled = model.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))
    padding = np.zeros((7, num_features - 1))
    prediction = scaler.inverse_transform(np.hstack([prediction_scaled.reshape(-1, 1), padding]))[:, 0]
    return prediction

# List to store predictions for all cities
all_cities_predictions = []

# Iterate through all unique city IDs
unique_city_ids = historical_weather['city_id'].unique()

for city_id in unique_city_ids:
    X, y, scaler = prepare_lstm_data(historical_weather, city_id)
    if X.shape[0] > 0:  # Ensure there is enough data for training
        model = train_lstm(X, y)
        num_features = X.shape[2]
        predictions = predict_next_7_days(model, X, scaler, num_features)
        all_cities_predictions.append({
            'city_id': city_id,
            'predictions': predictions
        })

# Display the predictions
all_cities_predictions

Epoch 1/20
50/50 [==============================] - 5s 28ms/step - loss: 0.0394 - val_loss: 0.0088
Epoch 2/20
50/50 [==============================] - 1s 13ms/step - loss: 0.0071 - val_loss: 0.0079
Epoch 3/20
50/50 [==============================] - 1s 12ms/step - loss: 0.0067 - val_loss: 0.0083
Epoch 4/20
50/50 [==============================] - 1s 12ms/step - loss: 0.0067 - val_loss: 0.0086
Epoch 5/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0063 - val_loss: 0.0074
Epoch 6/20
50/50 [==============================] - 1s 12ms/step - loss: 0.0063 - val_loss: 0.0070
Epoch 7/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0061 - val_loss: 0.0068
Epoch 8/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0059 - val_loss: 0.0081
Epoch 9/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0058 - val_loss: 0.0067
Epoch 10/20
50/50 [==============================] - 1s 12ms/step - loss: 0.0057 - val_loss: 0.0064
Epoch 11/

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [77]:
from sklearn.preprocessing import MinMaxScaler

# Function to prepare data for LSTM
def prepare_lstm_data(df, city_id, sequence_length=30):
    city_df = df[df['city_id'] == city_id].sort_values('date')
    features = ['avg_temp_c', 'min_temp_c', 'max_temp_c',
       'precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh',
       'temp_range', 'rolling_range_temp', 'avg_temp_rel', 'ema_temp',
       'cum_precip', 'temp_precip_interaction', 'temp_wind_interaction',
       'temp_anomaly', 'precip_anomaly', 'temp_precip_wind_interaction',
       'rolling_mean_temp', 'rolling_std_temp', 'rolling_sum_precip',
       'rolling_std_precip', 'rolling_mean_temp_14', 'rolling_std_temp_14',
       'rolling_sum_precip_14', 'rolling_std_precip_14',
       'rolling_mean_temp_30', 'rolling_std_temp_30', 'rolling_sum_precip_30',
       'rolling_std_precip_30', 'temp_deviation', 'temp_day_change',
       'heavy_precip','wind_dir_sin', 'wind_dir_cos',
       'temp_percentile', 'cum_avg_temp', 'mean_temp_c_month',
       'max_temp_c_month', 'min_temp_c_month', 'variance_temp_c_month',
       'skewness_temp_c', 'kurtosis_temp_c', 'heat_wave_duration',
       'cold_wave_duration', 'precipitation_event_duration', 'fourier_sin',
       'fourier_cos', 'dayofweek', 'quarter', 'month', 'season_sin_month',
       'season_cos_month', 'year', 'dayofyear', 'dayofmonth', 'season_sin_day',
       'season_cos_day', 'weekofyear', 'date_offset', 'season',
       'seasonal_mean_temp', 'avg_temp_c_lag_1', 'avg_temp_c_lag_2',
       'avg_temp_c_lag_3', 'avg_temp_c_lag_4', 'avg_temp_c_lag_5',
       'avg_temp_c_lag_6', 'avg_temp_c_lag_7', 'min_temp_c_lag_1',
       'min_temp_c_lag_3', 'min_temp_c_lag_7', 'max_temp_c_lag_1',
       'max_temp_c_lag_3', 'max_temp_c_lag_7', 'precipitation_mm_lag_1',
       'precipitation_mm_lag_3', 'precipitation_mm_lag_7',
       'avg_wind_dir_deg_lag_1', 'avg_wind_dir_deg_lag_3',
       'avg_wind_dir_deg_lag_7']
    
    city_df = city_df[features].dropna()
    
    # Check for large values or missing values
    if city_df.isnull().values.any():
        raise ValueError("Dataset contains missing values.")
    
    if not city_df.applymap(np.isreal).all().all():
        raise ValueError("Dataset contains non-numerical values.")

    # Scale numerical features
    scaler = MinMaxScaler()
    city_scaled = scaler.fit_transform(city_df)
    
    X, y = [], []
    for i in range(sequence_length, len(city_scaled) - 7):
        X.append(city_scaled[i-sequence_length:i])
        y.append(city_scaled[i:i+7, 0])  # Predict next 7 days of avg_temp_c

    X, y = np.array(X), np.array(y)
    return X, y, scaler

# Function to build and train LSTM model
def train_lstm(X_train, y_train):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(7))  # Predicting 7 days
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)
    return model

# List to store predictions for all cities
all_cities_predictions = []

# Iterate through all unique city IDs
unique_city_ids = historical_weather['city_id'].unique()

for city_id in unique_city_ids:
    try:
        X, y, scaler = prepare_lstm_data(historical_weather, city_id)
        if X.shape[0] > 0:  # Ensure there is enough data for training
            model = train_lstm(X, y)
            num_features = X.shape[2]
            # Assuming predict_next_7_days function remains the same
            predictions = predict_next_7_days(model, X, scaler, num_features)
            all_cities_predictions.append({
                'city_id': city_id,
                'predictions': predictions.tolist()  # Convert to list for JSON serialization
            })
    except ValueError as e:
        print(f"Skipping city_id {city_id}: {str(e)}")

# Display the predictions
all_cities_predictions


C:\Users\ASUS\AppData\Local\Temp\ipykernel_42752\1502393285.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not city_df.applymap(np.isreal).all().all():


Epoch 1/20
50/50 [==============================] - 6s 38ms/step - loss: 0.0511 - val_loss: 0.0099
Epoch 2/20
50/50 [==============================] - 1s 24ms/step - loss: 0.0073 - val_loss: 0.0090
Epoch 3/20
50/50 [==============================] - 1s 24ms/step - loss: 0.0068 - val_loss: 0.0097
Epoch 4/20
50/50 [==============================] - 1s 24ms/step - loss: 0.0064 - val_loss: 0.0107
Epoch 5/20
50/50 [==============================] - 1s 25ms/step - loss: 0.0064 - val_loss: 0.0081
Epoch 6/20
50/50 [==============================] - 1s 24ms/step - loss: 0.0063 - val_loss: 0.0093
Epoch 7/20
50/50 [==============================] - 1s 24ms/step - loss: 0.0060 - val_loss: 0.0080
Epoch 8/20
50/50 [==============================] - 1s 25ms/step - loss: 0.0057 - val_loss: 0.0079
Epoch 9/20
50/50 [==============================] - 1s 25ms/step - loss: 0.0056 - val_loss: 0.0075
Epoch 10/20
50/50 [==============================] - 1s 23ms/step - loss: 0.0056 - val_loss: 0.0078
Epoch 11/

C:\Users\ASUS\AppData\Local\Temp\ipykernel_42752\1502393285.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not city_df.applymap(np.isreal).all().all():


Epoch 1/20
50/50 [==============================] - 5s 30ms/step - loss: 0.0299 - val_loss: 0.0112
Epoch 2/20
50/50 [==============================] - 1s 16ms/step - loss: 0.0073 - val_loss: 0.0102
Epoch 3/20
50/50 [==============================] - 1s 17ms/step - loss: 0.0069 - val_loss: 0.0092
Epoch 4/20
50/50 [==============================] - 1s 16ms/step - loss: 0.0067 - val_loss: 0.0092
Epoch 5/20
50/50 [==============================] - 1s 16ms/step - loss: 0.0066 - val_loss: 0.0089
Epoch 6/20
50/50 [==============================] - 1s 16ms/step - loss: 0.0067 - val_loss: 0.0088
Epoch 7/20
50/50 [==============================] - 1s 16ms/step - loss: 0.0064 - val_loss: 0.0087
Epoch 8/20
50/50 [==============================] - 1s 16ms/step - loss: 0.0063 - val_loss: 0.0093
Epoch 9/20
50/50 [==============================] - 1s 17ms/step - loss: 0.0062 - val_loss: 0.0083
Epoch 10/20
50/50 [==============================] - 1s 15ms/step - loss: 0.0059 - val_loss: 0.0083
Epoch 11/

C:\Users\ASUS\AppData\Local\Temp\ipykernel_42752\1502393285.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not city_df.applymap(np.isreal).all().all():


...

C:\Users\ASUS\AppData\Local\Temp\ipykernel_42752\1502393285.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not city_df.applymap(np.isreal).all().all():


Epoch 1/20
50/50 [==============================] - 6s 48ms/step - loss: 0.0308 - val_loss: 0.0112
Epoch 2/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0099 - val_loss: 0.0111
Epoch 3/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0095 - val_loss: 0.0114
Epoch 4/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0093 - val_loss: 0.0108
Epoch 5/20
50/50 [==============================] - 1s 20ms/step - loss: 0.0092 - val_loss: 0.0114
Epoch 6/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0094 - val_loss: 0.0114
Epoch 7/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0091 - val_loss: 0.0108
Epoch 8/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0091 - val_loss: 0.0114
Epoch 9/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0090 - val_loss: 0.0116
Epoch 10/20
50/50 [==============================] - 1s 20ms/step - loss: 0.0089 - val_loss: 0.0109
Epoch 11/

C:\Users\ASUS\AppData\Local\Temp\ipykernel_42752\1502393285.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not city_df.applymap(np.isreal).all().all():


Epoch 1/20
50/50 [==============================] - 5s 34ms/step - loss: 0.0240 - val_loss: 0.0085
Epoch 2/20
50/50 [==============================] - 1s 20ms/step - loss: 0.0108 - val_loss: 0.0063
Epoch 3/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0105 - val_loss: 0.0070
Epoch 4/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0105 - val_loss: 0.0067
Epoch 5/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0101 - val_loss: 0.0073
Epoch 6/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0100 - val_loss: 0.0074
Epoch 7/20
50/50 [==============================] - 1s 20ms/step - loss: 0.0098 - val_loss: 0.0078
Epoch 8/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0098 - val_loss: 0.0062
Epoch 9/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0095 - val_loss: 0.0076
Epoch 10/20
50/50 [==============================] - 1s 19ms/step - loss: 0.0098 - val_loss: 0.0066
Epoch 11/

[{'city_id': 1,
  'predictions': [8.52294921875,
   9.171687403321267,
   9.82975294291973,
   9.7690320789814,
   9.81543739438057,
   9.728157234191894,
   9.184134268760682]},
 {'city_id': 2,
  'predictions': [14.070727610588074,
   14.129808855056764,
   14.018131947517396,
   13.678805220127106,
   13.973375523090363,
   13.703468906879426,
   14.25259439945221]},
 {'city_id': 3,
  'predictions': [26.367590165138246,
   26.3704376578331,
   26.499052220582964,
   26.465672141313554,
   26.418982893228534,
   26.088674342632295,
   26.222444492578507]},
 {'city_id': 4,
  'predictions': [1.0645414203405368,
   0.5336629658937442,
   -0.5404298692941678,
   0.588764479756354,
   1.1322638452053058,
   0.26937038302421445,
   0.5969731569290149]},
 {'city_id': 5,
  'predictions': [25.09335062503815,
   24.261774384975435,
   24.579080295562747,
   23.775943756103516,
   24.048203361034396,
   24.28273210525513,
   24.981254148483277]},
 {'city_id': 7,
  'predictions': [18.898175287246

In [78]:
len(all_cities_predictions)

99

In [79]:
results = []

for city_prediction in all_cities_predictions:
    city_id = city_prediction['city_id']
    predictions = city_prediction['predictions']
    for i, pred in enumerate(predictions):
        results.append({'city_id': city_id, 'day': i + 1, 'predicted_avg_temp_c': pred})

df_predictions = pd.DataFrame(results)

In [80]:
df_predictions

,city_id,day,predicted_avg_temp_c
0,1,1,8.522949
1,1,2,9.171687
2,1,3,9.829753
3,1,4,9.769032
4,1,5,9.815437
...,...,...,...
688,112,3,21.520502
689,112,4,21.630846
690,112,5,21.606938
691,112,6,22.063850


In [ ]:
# pip install --upgrade pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [81]:
import pandas as pd

submission = pd.DataFrame({
    'submission_ID': test_data['submission_ID'],
    'avg_temp_c': df_predictions['predicted_avg_temp_c']
})

# Save submission to CSV file
submission.to_csv('submission_LSTM_7.csv', index=False)